In [1]:
import pandas as pd
from ALLCools.mcds import MCDS
from scipy.stats import f_oneway, ttest_ind, kruskal
import joblib
import anndata
from wmb import mm10
import pingouin as pg
import xarray as xr
import joblib
import numpy as np
import glob
from matplotlib import pyplot as plt
import seaborn as sns

plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi']=300
plt.rcParams['pdf.fonttype']=42

gene_name_to_id = joblib.load('240112_DMG_CEF/gene_name_to_id_dict')
gene_id_to_name = joblib.load('240112_DMG_CEF/gene_id_to_name_dict')

In [3]:
_gender = 'Male'
mc_type = 'CGN'
p_cutoff = 1e-5
ct = 'Oligo_NN'

In [69]:
total_data = xr.open_dataset(f'{mc_type}.{_gender}.AgingMajorType.Region.Anova.Result.zarr')

/home/qz/miniconda3/envs/allcools/lib/python3.7/site-packages/xarray/backends/plugins.py:114: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/qz/miniconda3/envs/allcools/lib/python3.7/site-packages/xarray/backends/plugins.py:114: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


In [70]:
rna_df = pd.read_csv(f"DEGs/{ct].csv", index_col =0)
rna_df['log2(aged/young)'] = -rna_df['avg_log2FC']


In [71]:
use_region_cts = ['DG_Glut.Anterior_Hippocampus','DG_Glut.Posterior_Hippocampus',]

In [72]:
dmg_num_df = {}
for region_ct in use_region_cts:
    ct, region = region_ct.split('.')
    tmpdf = total_data.sel(region_ct= region_ct)['anova_results'].to_pandas()
    _up_tmpdf = tmpdf[(tmpdf['8wk'] < tmpdf['9mo']) & (tmpdf['9mo'] < tmpdf['18mo'])]
    _down_tmpdf = tmpdf[(tmpdf['8wk'] > tmpdf['9mo']) & (tmpdf['9mo'] > tmpdf['18mo'])]
    tmpdf= pd.concat([_up_tmpdf, _down_tmpdf])
    tmpdf = tmpdf[tmpdf['corrected_pvalue'] < p_cutoff]
    tmpdf = tmpdf.sort_values('corrected_pvalue')
    dmg_num_df[region_ct] = tmpdf

In [37]:
for gene_id in tmpdf.index:
    if gene_id_to_name[gene_id].startswith('Gm') == False and gene_id_to_name[gene_id].startswith('Mir') == False:
        if gene_id_to_name[gene_id] in rna_df.index:
            print(f"{gene_id}:{gene_id_to_name[gene_id]}")

ENSMUSG00000029406.15:Pitpnm2
ENSMUSG00000001520.12:Nrip2
ENSMUSG00000104585.1:4921511C10Rik
ENSMUSG00000079484.12:Phyhd1
ENSMUSG00000076617.9:Ighm
ENSMUSG00000096910.2:Zfp955b
ENSMUSG00000049928.15:Glp2r
ENSMUSG00000112280.1:A830082N09Rik
ENSMUSG00000035337.3:Uchl4
ENSMUSG00000100281.1:4930422M22Rik
ENSMUSG00000084792.1:1700056N10Rik
ENSMUSG00000085030.1:2810455O05Rik
ENSMUSG00000027001.10:Dusp19
ENSMUSG00000094441.2:Zfp955a


In [78]:
goi = gene_id_to_name['ENSMUSG00000027001.10']
goi

'Dusp19'

In [79]:
rna_df.loc[[goi]]

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,log2(aged/young)
Dusp19,2.371601e-07,0.915022,0.019,0.01,0.00616,-0.915022


In [4]:
goi = 'C4b'

In [23]:
all_df =[] 

for fp in glob.glob(f"Regional.aDEG.2/{ct}.*/stats.hdf"):
    deg_df = pd.read_hdf(fp)
    if goi in deg_df.index:
        deg_df = pd.DataFrame(deg_df.loc[goi, ['fdr', 'log2(18mo/2mo)']])
        deg_df['region_ct'] = fp.split('/')[-2]
        all_df.append(deg_df)

all_df = pd.concat(all_df)

## check astro

In [2]:
import pandas as pd
from ALLCools.mcds import MCDS
from scipy.stats import f_oneway, ttest_ind, kruskal
import joblib
import anndata
from wmb import mm10
import pingouin as pg
import xarray as xr
import joblib
import numpy as np
import glob
from matplotlib import pyplot as plt
import seaborn as sns

plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi']=300
plt.rcParams['pdf.fonttype']=42

gene_name_to_id = joblib.load('240112_DMG_CEF/gene_name_to_id_dict')
gene_id_to_name = joblib.load('240112_DMG_CEF/gene_id_to_name_dict')

In [3]:
_gender = 'Male'
mc_type = 'CGN'
p_cutoff = 1e-5
ct = 'Oligo_NN'

/home/qz/miniconda3/envs/allcools/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/home/qz/miniconda3/envs/allcools/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [4]:
merfish_genes = pd.read_csv('annotated_FinalGeneList_ecker-amb-500_CP1145.csv', index_col =0)['Vizgen Gene'].values

In [22]:
tmpdf = pd.read_csv(f"result/{_gender}.{mc_type}.AllGene.Anova.Result.csv")
tmpdf = tmpdf[tmpdf['celltype'].isin(['Astro-TE NN','Astro-NT NN'])]
tmpdf['gene_name'] = tmpdf['genes'].map(gene_id_to_name)

# _up_tmpdf = tmpdf[(tmpdf['8wk'] < tmpdf['9mo']) & (tmpdf['9mo'] < tmpdf['18mo'])]
# _down_tmpdf = tmpdf[(tmpdf['8wk'] > tmpdf['9mo']) & (tmpdf['9mo'] > tmpdf['18mo'])]
# tmpdf= pd.concat([_up_tmpdf, _down_tmpdf])
# dmg_df = tmpdf[tmpdf['pvalue'] < p_cutoff]


/home/qz/miniconda3/envs/allcools/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
# merfish_dmg_df= dmg_df[dmg_df['gene_name'].isin(merfish_genes)]
# merfish_dmg_df.sort_values('corrected_pvalue')

In [27]:
goi = 'Camta1'

In [28]:
merfish_dmg_df[merfish_dmg_df['gene_name'] == goi]

,genes,pvalue,corrected_pvalue,18mo,8wk,9mo,gene_length,celltype,methylation_change,abs_change,trend,gene_name
25972,ENSMUSG00000014592.20,4.399345e-01,1.000000e+00,0.784441,0.781003,0.787327,944554,Astro-NT NN,0.003438,0.003438,NaN,Camta1
57075,ENSMUSG00000014592.20,5.310808e-24,2.484745e-19,0.761497,0.738208,0.754551,944554,Astro-TE NN,0.023289,0.023289,NaN,Camta1


In [34]:
astro_nt_df.loc[[goi]]

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
Camta1,2.241959e-23,0.235689,0.468,0.329,5.823263e-19


In [17]:
def load_df(fp):
    df = pd.read_csv(fp, index_col = 0)
    df = df.loc[df.index.isin(merfish_genes)]
    deg_df = df[(df['p_val_adj'] < 0.01) & (abs(df['avg_log2FC']) > 0.1)]
    return df, deg_df

In [18]:
astro_te_df,astro_te_deg_df  = load_df(f"DEGs/Astro-TE_NN.csv")
astro_nt_df, astro_nt_deg_df = load_df(f"DEGs/Astro-NT_NN.csv")

In [ ]:
# Pde1a, Shisa9, Camta1 down in NT

In [23]:
goi_list = ['Pde1a', 'Shisa9', 'Camta1']

In [25]:
tmpdf[tmpdf['gene_name'].isin(goi_list)]

,genes,pvalue,corrected_pvalue,18mo,8wk,9mo,gene_length,celltype,methylation_change,abs_change,trend,gene_name
25972,ENSMUSG00000014592.20,4.399345e-01,1.000000e+00,0.784441,0.781003,0.787327,944554,Astro-NT NN,0.003438,0.003438,NaN,Camta1
26584,ENSMUSG00000022494.15,5.729657e-01,1.000000e+00,0.793323,0.790485,0.781958,286789,Astro-NT NN,0.002837,0.002837,NaN,Shisa9
39667,ENSMUSG00000059173.19,4.549861e-02,1.000000e+00,0.850308,0.851461,0.865136,295005,Astro-NT NN,-0.001153,0.001153,NaN,Pde1a
57075,ENSMUSG00000014592.20,5.310808e-24,2.484745e-19,0.761497,0.738208,0.754551,944554,Astro-TE NN,0.023289,0.023289,NaN,Camta1
57687,ENSMUSG00000022494.15,6.502015e-02,1.000000e+00,0.799616,0.787815,0.795259,286789,Astro-TE NN,0.011801,0.011801,NaN,Shisa9
70743,ENSMUSG00000059173.19,1.824005e-02,5.718520e-01,0.824481,0.815570,0.820984,295005,Astro-TE NN,0.008910,0.008910,NaN,Pde1a
